In [7]:
import torch
from scipy import sparse
import numpy as np

In [8]:
num_angle = 179
num_offset = 1
num_spotx = 130
chord_length = 'map'
RSP_shape = [130, 130]

In [9]:
mlp = sparse.load_npz(f'../../Data/simple_pCT/MLP/MLP_angles{num_angle}_offset{num_offset}_spotx{num_spotx}_{chord_length}_{RSP_shape[0]}_{RSP_shape[1]}.npz')
wepl = np.load(f'../../Data/simple_pCT/WEPL/WEPL_angles{num_angle}_offset{num_offset}_spotx{num_spotx}_{chord_length}_{RSP_shape[0]}_{RSP_shape[1]}.npy').flatten()

In [10]:
obj = np.zeros(RSP_shape, dtype='bool')
for i in range(RSP_shape[0]):
    for j in range(RSP_shape[1]):
        # a circle is used as a valid volume
        vec = np.array([i, j])
        center = np.array(RSP_shape) // 2 
        
        obj[i,j] = True if (i - center[0])**2 / center[0]**2 + (j - center[1])**2 / center[1]**2 <= 1 else False

obj = obj.flatten()

In [52]:
A_norm = sparse.linalg.norm(mlp, axis=1)**2
valid = A_norm > 1e-9
A_norm = A_norm[valid].astype(np.float32)

S_array = np.array([mlp[:,i].count_nonzero() for i in range(mlp.shape[1])])
S_array[ S_array <= 0 ] = 1
S_array = 1 / S_array
S_array = S_array.astype(np.float32)

In [70]:
x = torch.rand((np.prod(RSP_shape), 1), dtype=torch.float32)

lamb = 0.5
coo = mlp[valid,:].tocoo()
indices = np.mat([coo.row, coo.col])
A = torch.sparse_coo_tensor(indices, coo.data.astype(np.float32), coo.shape)
b = torch.tensor(wepl[valid].astype(np.float32))
S = torch.tensor(S_array * lamb)

def dropSetp(x, A, b, S):

    # print('S:', S.shape)
    Ax = torch.reshape(torch.sparse.mm(A, x), (-1,))
    # print('Ax:', Ax.shape)
    # print('b:', b.shape)

    residuals = b - Ax
    # print('residuals:', residuals.shape)
    factor = residuals / A_norm
    # print('factor:', factor.shape)
    temp1 = torch.transpose(A, 0, 1)
    # print('temp1:', temp1.shape)
    temp2 = torch.matmul(temp1, factor)
    # print('temp2:', temp2.shape)
    temp3 = S * temp2
    # print('temp3:', temp3.shape)

    return x + temp3, residuals

hist = []
for iter in range(2000):
    x, res = dropSetp(x, A, b, S)

#ft_jacobian = torch.func.jacrev(dropSetp, argnums=2)(x, A, b, S, lamb)


RuntimeError: The size of tensor a (23101) must match the size of tensor b (390406900) at non-singleton dimension 0